<a href="https://colab.research.google.com/github/coldsober-irene/NLP-fellowship/blob/main/week1Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ASSIGNMENT II** : scrap igihe.com from January to present

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
from google.colab import drive
import numpy as np
import pandas as pd

In [2]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
unsortedLinks = []

for month in range(1, 11):
  for day in range(1, 32):
    l = "http://archive.org/wayback/available?url=igihe.com&timestamp=2022{:02d}{:02d}".format(month, day)
    response = requests.get(l).json()
    if  response['archived_snapshots']:
      unsortedLinks.append(response['archived_snapshots']['closest']["url"])

In [ ]:
len(unsortedLinks)

In [ ]:
# GET ALL ARTICLE AVAILABLE IN EVERY LINK


In [ ]:
arr = np.array(unsortedLinks)
series = pd.Series(arr)
unsortedLinks = series.unique()

In [ ]:
serious_links = {}


In [ ]:
# link = "https://web.archive.org/web/20220615035616/https://www.igihe.com/amakuru/"
def process_links(link):
  page = requests.get(link)
  soup = bs(page.text, "html.parser")
  
  b = soup.body
  get_titles = [link+re.sub("amakuru/","", a['href']) for span in b.find_all("span", class_ = "homenews-title") for a in span.find_all("a")]
  
  # get only links whose date is in current year
  pattern = re.compile(r"\d{8,}?")
  found_date = pattern.search(link).group()
  if found_date[3] != str(2):
    pass
  else:
    file_title = f"igihe-{found_date[0:4]}-{found_date[4:6]}-{found_date[6:8]}"
    try:
      for title in get_titles:
        serious_links[file_title].append(title)
    except KeyError:
      serious_links[file_title] = []
      for title in get_titles:
        serious_links[file_title].append(title)

In [ ]:
for link in unsortedLinks:
  process_links(link)

In [ ]:
serious_links

In [ ]:
def writeToFile(paragraphs, file):
  with open(file, "a+") as f:
    for line in paragraphs:
      f.write(line)

In [ ]:
def get_page():
  count = 1
  for key in list(serious_links.keys()):
    for link in set(serious_links[key]):
      p = requests.get(link).content
      soup = bs(p, "html.parser")
      spans = soup.find_all("div", class_= "fulltext margintop10")
      paragraphs = [p.get_text() for span in spans for p in span.find_all("p")]
      writeToFile(paragraphs = paragraphs, file = f"/content/drive/MyDrive/NLP fellowship/week1 tasks/{key}.txt")
    print(f"file {count} created")
    count += 1

In [ ]:
# FINISH THE PROGRAM
get_page()

**ASSIGNMENT III** : extract text from 5 pdf documents provided and write them in a single file

In [ ]:
!pip install textract


In [ ]:
!pip install PyPDF2

In [ ]:
 import os
 import textract, PyPDF2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# GET THE PDF FILES 
os.chdir("/content/drive/MyDrive/NLP fellowship/files")
PDFs = [file for file in os.listdir() if "Hansard" in file]

In [ ]:
PDFs

In [ ]:
def remove_firstPage(pdfs):
  pages = {}
  temp = PyPDF2.PdfFileWriter()
  for pdf in pdfs:
    pages[pdf] = []

  for pdf in pdfs:
    reader = PyPDF2.PdfFileReader(pdf)
    allowed_pages = reader.getNumPages()
    for page_number in range(1, allowed_pages-1):
      # pages[pdf].append(reader.getPage(page_number))
      temp.add_page(reader.getPage(page_number))

  return temp


In [ ]:
pages = remove_firstPage(PDFs)

In [ ]:
# create template pdf file
with open("/content/temp.pdf", "wb") as f:
  pages.write(f)

In [ ]:
# EXTRACT EACH PDF
def extract(file, new_destination = "PDFsExtracted.txt"):
  text = textract.process(file).decode("utf-8")
  lines = text.split("\n")
  with open(new_destination, "a+") as f:
    for line in lines:
      if line.strip():
        f.write(line+"\n")
  with open(new_destination, "a+") as f:
    f.write("\n")
  
extract("/content/temp.pdf")

**END OF THE WEEK ASSIGNMENT** data cleaning 

In [33]:
engKinyFile = "/content/drive/MyDrive/NLP fellowship/week1 tasks/Eng_Kin-Paralleldata (1).csv" 

In [55]:
# READ THE FILE
df = pd.read_csv(engKinyFile)
kiny = df['Kinyarwanda']
eng = df["English"]

In [ ]:
kiny

In [57]:
# NORMALIZING THE FILE
kiny = kiny.str.lower()
eng = eng.str.lower()

In [58]:
# REMOVE THE WHITE SPACES
kiny = kiny.str.strip()
eng = eng.str.strip()

In [ ]:
print(kiny)

In [60]:
kiny_list = list(kiny)
eng_list = list(eng)

In [ ]:
kiny_list

In [9]:
import string
punc = string.punctuation
print(punc)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [93]:
# REMOVE URLs and EMAILS
pattern = re.compile("\w+@[^\s\.]{3,}|([a-z]+\.[\w]{3,})|([^\s]+(\.net|\.org|\.com))|www\..*|htt\.*|[^\s]+@[^\s]+")
found = [pattern.findall(line) for line in eng_list if pattern.findall(line)]
EngNo_urls = [pattern.sub("",line) for line in eng_list]
KinyNo_urls = [pattern.sub("",line) for line in kiny_list]

In [ ]:
# EngNo_urls
found
# KinyNo_urls

In [72]:
# REMOVE PANCTUATIONS 
panc_pattern = re.compile("[^\w\s]+")
EngNo_punc = [panc_pattern.sub("", line) for line in EngNo_urls]
KinyNo_punc = [panc_pattern.sub("", line) for line in KinyNo_urls]

In [ ]:
KinyNo_punc

In [75]:
# REMOVE NUMBERS
dig_pattern = re.compile("\d+")
EngNo_dig = [dig_pattern.sub("", line) for line in EngNo_punc]
KinyNo_dig = [dig_pattern.sub("", line) for line in KinyNo_punc]

In [ ]:
KinyNo_dig

In [ ]:
# remove english words in kinyarwanda
eng_pattern = re.compile("\w+[b-df-hj-np-tv-xz]\b")
all_engWords = [print(line) for line in KinyNo_dig]

In [ ]:
all_engWords